<a href="https://colab.research.google.com/github/aliaghababaee/SLO_Asieh/blob/main/Optuna_merged_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!unzip /content/drive/MyDrive/merged_data_final2.zip

Archive:  /content/drive/MyDrive/merged_data_final2.zip
 extracting: test_merged.pkl         
 extracting: test_merged_label.pkl   
 extracting: train_merged.pkl        
 extracting: train_merged_label.pkl  


In [13]:
!unzip /content/drive/MyDrive/new_test_merged_data.zip

Archive:  /content/drive/MyDrive/new_test_merged_data.zip
replace test_merged.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test_merged.pkl         
replace test_merged_label.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test_merged_label.pkl   


In [4]:
!pip install optuna
!pip install optuna-integration
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.0 MB/s eta 0:00:00


In [16]:
### SLO model
from tensorflow.keras.models import Model
slo_model =  tf.keras.applications.ResNet101(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3),
    )

for layer in slo_model.layers:
    layer._name = 'SLO_' + layer.name

new_model = Model(inputs=slo_model.input, outputs=slo_model.output)

new_model.save('slo_model_resnet101.h5')

171446536/171446536 [==============================] - 9s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
####################### state of the arts ##############################
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.layers import Input

def classifier (trial):

    ### SLO model
    slo_model = load_model('slo_model_resnet101.h5')

    for layer in slo_model.layers:
        layer.trainable = False

    ### OCT model

    oct_model =  tf.keras.applications.ResNet101(
        weights='imagenet',
        include_top=False,
        input_shape=(60,256,3),
        )

    for layer in oct_model.layers:
        layer.trainable = False


    inputs_slo = tf.keras.layers.Input((128,128,3))
    slo_output = tf.keras.layers.Flatten()(slo_model(inputs_slo))


    inputs_oct = tf.keras.layers.Input((60,256,3))
    oct_output = tf.keras.layers.Flatten()(oct_model(inputs_oct))


    model = tf.keras.layers.Concatenate(axis=-1)([slo_output, oct_output])


    dropout_l0 = trial.suggest_float("dropout_l0", 0, 0.7,step=0.1)

    model = tf.keras.layers.Dropout(dropout_l0)(model)


    n_layers = trial.suggest_int("n_layers", 1, 5)


    for i in range(n_layers):

        n_units = trial.suggest_int("n_units_l{}".format(i), 8, 4096, log = True)

        model = tf.keras.layers.Dense(n_units, activation = 'relu')(model)

        dropout = trial.suggest_float("dropout_l{}".format (i+1), 0, 0.7,step=0.1)

        model = tf.keras.layers.Dropout(rate = dropout)(model)


    outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(model)

    lr = trial.suggest_float ('lr', 1e-5, 1e-3, log = True)

    my_optimizer = tf.keras.optimizers.Adam (learning_rate= lr)

    model_merged = tf.keras.Model([inputs_slo, inputs_oct] , outputs)


    model_merged.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics='accuracy')

    return model_merged

In [14]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold (n_splits = 5, shuffle = True, random_state = 42)
import keras
import pickle
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def preparing(x, y):

    data_slo  = []
    data_oct  = []
    label     = []
    for i in x:
      for j in range(len(x[i])):
          data_slo.append(np.array(x[i][j][0])/255)
          data_oct.append(np.array(x[i][j][1])*255)
          label.append(y[i])
    data_slo = np.reshape(data_slo, np.shape(data_slo))
    data_oct = np.reshape(data_oct, np.shape(data_oct))
    return data_slo, data_oct, np.array(label)



def objective (trial):

  keras.backend.clear_session()

##################??????????????????????????????????????????????

  datagen_oct = ImageDataGenerator(
  rotation_range= 5, # rotation
  zoom_range= 0.2,
  vertical_flip= True , # vertical fli
  fill_mode='constant',
  data_format='channels_last',
  cval=0,
    )

  datagen_slo = ImageDataGenerator(
  rotation_range= 5, # rotation
  width_shift_range= [-30, 30], # horizontal shift
  height_shift_range= [-5, 5] , # vertical shift
  zoom_range= 0.2,
  vertical_flip= True , # vertical flip
  brightness_range= [0.2, 1.5]
    )


  images_train = pickle.load(open("/content/train_merged.pkl", 'rb'))
  labels_train = pickle.load(open("/content/train_merged_label.pkl", 'rb'))

  train_index, val_index = next (skf.split (images_train, list(labels_train.values())))

  x_train = {i: images_train[list(images_train.keys())[i]] for i in train_index}
  x_valid = {i: images_train[list(images_train.keys())[i]] for i in val_index}

  y_trainn = {i: labels_train[list(labels_train.keys())[i]] for i in train_index}
  y_validd = {i: labels_train[list(labels_train.keys())[i]] for i in val_index}


  ################## preparing

  x_train_slo, x_train_oct, y_train = preparing(x_train,y_trainn)
  x_valid_slo, x_valid_oct, y_valid = preparing(x_valid,y_validd)

  ################# Augmentation

  ############ slo
  x_train_aug =np.zeros_like(x_train_slo, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)


  for i in range(len(x_train_slo)):

    x1= x_train_slo[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_slo.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_slo = np.concatenate ((x_train_slo, x_train_aug), axis=0)

  y_train_slo = np.concatenate ((y_train, y_train_aug), axis = 0)

  ############ OCT
  x_train_aug =np.zeros_like(x_train_oct, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)

  for i in range(len(x_train_oct)):

    x1= x_train_oct[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_oct.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_oct = np.concatenate ((x_train_oct, x_train_aug), axis=0)

  y_train_oct = np.concatenate ((y_train, y_train_aug), axis = 0)


  indices = np.random.permutation (len (x_train_slo))
  x_train_slo = x_train_slo [indices]
  y_train_slo = y_train_slo [indices]

  x_train_oct = x_train_oct [indices]
  y_train_oct = y_train_oct [indices]


  x_train_slo = np.repeat (x_train_slo, repeats = 3, axis = 3)

  x_train_oct = np.repeat (x_train_oct, repeats = 3, axis = 3)

  x_valid_slo = np.repeat (x_valid_slo, repeats = 3, axis = 3)

  x_valid_oct = np.repeat (x_valid_oct, repeats = 3, axis = 3)


  ####################################################################
  # classification via my model
  ####################################################################
  batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128])

  model = classifier (trial)


  # Generate our trial model.


  model.fit([x_train_slo, x_train_oct],
            np.asarray(y_train_oct, dtype=np.float64),
            batch_size= batch_size,
            epochs=50,
            callbacks=[TFKerasPruningCallback(trial, "val_accuracy"), EarlyStopping(patience=10, verbose=1),
            ReduceLROnPlateau(factor=0.1, patience=10, min_lr=1e-6),
            ModelCheckpoint(f'slo_oct.h5', verbose=1, save_best_only=True, save_weights_only=True)],
            validation_data=([x_valid_slo, x_valid_oct] , np.asarray(y_valid, dtype=np.float64)),
            )
  model.load_weights(f'slo_oct.h5')
  score = model.evaluate ([x_valid_slo, x_valid_oct], np.asarray(y_valid, dtype=np.float64), verbose = 1)

  return score [1]


In [19]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())

study.optimize(objective, n_trials=100)

[I 2024-02-28 10:22:58,239] A new study created in memory with name: no-name-b5b4dd1b-78b6-43dc-a09a-0b992fc6cbdc
[W 2024-02-28 10:23:14,703] Trial 0 failed with parameters: {'batch_size': 16} because of the following error: RuntimeError('pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>\n\nAt:\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/core.py(42): __init__\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/gen_math_ops.py(477): add_v2\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/math_ops.py(4098): add\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py(1260): op_dispatch_handler\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py(150): error_handler\n  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/weak_tensor_ops.py(142): wrapper\n  /usr/

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/core.py(42): __init__
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/gen_math_ops.py(477): add_v2
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/math_ops.py(4098): add
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py(1260): op_dispatch_handler
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py(150): error_handler
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/weak_tensor_ops.py(142): wrapper
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/stateless_random_ops.py(406): stateless_random_uniform
  /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py(1260): op_dispatch_handler
  /usr/local/lib/p

In [18]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  15
  Number of pruned trials:  9
  Number of complete trials:  5
Best trial:
  Value:  1.0
  Params: 
    batch_size: 16
    dropout_l0: 0.6000000000000001
    n_layers: 2
    n_units_l0: 526
    dropout_l1: 0.6000000000000001
    n_units_l1: 676
    dropout_l2: 0.7
    lr: 0.00024271014447055212
